In [ ]:

####在diorpy python 中写出 R   就是用临时tmp.h5 调用R来写出RDS
def write_rds(adata: Union[str, None] = None,
	          file: Union[str, None] = None,
             object_type:str = 'seurat',
             assay_name: str = 'RNA'
            ) -> None:
    rfile = re.sub('.rds','_tmp.h5',file)
    write_h5(adata=adata, file=rfile, assay_name=assay_name)
    current_path = os.path.abspath(__file__)
    diorc_file= os.path.abspath(os.path.dirname(current_path) + os.path.sep + ".") + '/R/diorC.R'
    os.system('Rscript ' + diorc_file +' -r '+ rfile +' -t '+ object_type + ' -a '+assay_name)
    return 


library(getopt)
library(dior)
spec <- matrix(
  c('readfile','r',1,'character','Reading the h5 file',
    'targetobject', 't',1,'character','The single-cell data object which supprots Seurat and SingleCellExperiment',
    'assay_name','a',1,'character','The primary data types, such as scRNA data or spatial data'
  ),
  byrow = TRUE, ncol =5
)
opt <- getopt(spec)
# read the rds file
data <- read_h5(file = opt$readfile, assay.name = opt$assay_name, target.object = opt$targetobject)
saveRDS(data, file = gsub('_tmp.h5', '.rds', opt$readfile))


In [ ]:
####在diorpy python 中读入出 R   就是调用R来写出tmp.h5

# read the R rds file 
def read_rds(file: Union[str, None] = None,
             object_type:str = 'seurat',
             assay_name: str = 'RNA'
            ) -> anndata.AnnData:

    # osr = os.path.join(os.path.dirname(__file__), '/R/diopyR.R')
    current_path = os.path.abspath(__file__)
    diopyr_file= os.path.abspath(os.path.dirname(current_path) + os.path.sep + ".") + '/R/diopyR.R'
    os.system('Rscript ' + diopyr_file +' -r '+ file +' -t '+ object_type+' -a '+assay_name)
    tmp = re.sub('.rds', '_tmp.h5', file)
    adata = read_h5(file =tmp, assay_name = assay_name)
    return adata

#--- To be continues



library(getopt)
library(dior)
spec <- matrix(
  c('readfile','r',1,'character','Reading the rds file',
    'targetobject', 't',1,'character','The single-cell data object which supprots Seurat and SingleCellExperiment'
    'assayname', 'a',1,'character','Denoting which omics data to save'
  ),
  byrow = TRUE, ncol =5
)
opt <- getopt(spec)
# read the rds file
data <- readRDS(opt$readfile)
ind <- gregexpr('/', opt$readfile)
rdir <- substr(opt$readfile, start = 1, stop = max(ind[[1]]))
fname <- substr(opt$readfile, start = max(ind[[1]])+1, stop = nchar(opt$readfile))
wname <- gsub('.rds', '_tmp.h5', fname)
write_h5(data = data, object.type = opt$targetobject ,file = paste0(rdir, wname),
         assay.name = opt$assayname, save.graphs = TRUE, save.scale = FALSE)

In [ ]:

#####在R （dior）中 写出到 R (rds文件) 则是调用python


write_h5 <- function(data, file, object.type = 'seurat', assay.name = 'RNA', save.graphs = TRUE, save.scale=FALSE){
  if(object.type == 'seurat'){
    seurat_write_h5(seurat = data, file = file, assay.name = assay.name, save.graphs = save.graphs, save.scale = save.scale)
  }
  if(object.type == 'singlecellexperiment'){
    sce_write_h5(sce = data, file = file, assay.name = assay.name)
  }
}


#--- read_h5 part

#' The read the some data in the h5 file
#'
#' @param file the h5 file
#' @param groups the h5 groups
#' @importFrom hdf5r H5File h5attr
#' @return The list contains the core data for single-cell information
#'
#' @export
read_h5part <- function(file, groups){
  h5 <- H5File$new(filename = file, mode = 'r')
  h5_list <- list()
  for(h in groups){
    h5_list[[h]] <- switch(h, data = to_data_(h5),
                               obs = to_obs_(h5),
                               var = to_var_(h5),
                               dimR = seurat.to_dimr_(h5),
                               layers = to_layers_(h5),
                               graphs = to_graphs_(h5),
                               spatial = seurat.to_spatial_(h5),
                               uns = to_uns_(h5))
  }
  return(h5_list)
}

#' reading the h5ad file
#'
#' @param file the file name
#' @param assay_name the assay_name
#' @param target.object the target.object name
#'
#' @export
read_h5ad <-function(file,
                     assay_name,
                     target.object){

  rfile = paste0(system.file(package = 'dior'), '/python/diorPy.py')
  if(file.exists(rfile)){
    rfile = rfile
  }else{
    rfile = paste0(system.file(package = 'dior'), '/inst/python/diorPy.py')
  }
  py_script = paste0('python ', rfile, ' -f ', file, ' -a ', assay_name)
  system(py_script)
  data = read_h5(file = gsub('.h5ad', '_tmp.h5', file), target.object = target.object)   #####读入h5ad 转过来的临时文件！
  return(data)
}





















# python diopy 
import argparse
import scanpy as sc
from diopy import output
import re

# set the parser
parser = argparse.ArgumentParser(description='R read the h5ad ', epilog="diorpy -f file.h5ad -t seurat")
parser.add_argument('-f', '--file', dest = 'file', help = 'The h5ad file', type = str, required = True)
parser.add_argument('-a', '--assay_name', dest = 'assay_name', help = 'The data type assay', type = str, required = True)
args = parser.parse_args()

# save the h5 file
adata = sc.read(args.file)
output.write_h5(adata = adata,file=re.sub('.h5ad', '_tmp.h5', args.file), assay_name=args.assay_name)#h5ad 转过的临时文件！